In [11]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [12]:
hdb_price = pd.read_csv('hdb_price.csv')
hdb_intervals = pd.read_csv('hdb_intervals.csv')

In [15]:
import json
from io import StringIO
import boto3
import os
import pandas as pd

def cal_price(start_time, end_time, carpark_id):
    price = 0
    while start_time < end_time:
        day = start_time.weekday() + 1
        hour = start_time.hour * 2
        mins = (start_time.minute//30 )
        if hour < 10:
            hour_mins = '0' + str(hour + mins)
        else:
            hour_mins = str(hour + mins)
        interval = hdb_intervals[hdb_intervals.id == carpark_id][str(day) + hour_mins]
        price += hdb_price[hdb_price.id == carpark_id][str(day) + hour_mins].iloc[0]
        start_time += timedelta(minutes = int(interval.iloc[0]))
    return round(price, 2)

def lambda_handler(event, context):
    try:
        # Get variables from event
        carparkid = event.get('id')
        stime = event.get('start_time')
        etime = event.get('end_time')
        
        # Set client to get file from S3
        s3_client = boto3.client('s3')
        response1 = s3_client.get_object(Bucket='aws-build-on-parking',
                                        Key='hdb_price.csv')
        hdb_price = response1["Body"]
    
        # Load csv as a Pandas Dataframe
        df = pd.read_csv(csv_file, index_col=0, low_memory=False)
    
        # Create groupby Dataframe
        df_groupby = pd.DataFrame(df.groupby(GROUP)[
            COLUMN].mean().sort_values(ascending=False)).round(2)
    
        # Save the Dataframe to the same S3 BUCKET
        write_dataframe_to_csv_on_s3(df_groupby, OUTPUT, BUCKET)
    
        return {
            'statusCode': 200,
            'message':'Success!'}
    except:
        return {
            'statusCode': 400,
            'body': 'Error, bad request!'}

In [17]:
s3_client = boto3.client('s3')
response1 = s3_client.get_object(Bucket='aws-build-on-parking',
                                Key='hdb_price.csv')
hdb_price = response1["Body"]

NoCredentialsError: Unable to locate credentials